In [1]:
import scvi
import scanpy as sc
import os
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from cfp.external import CFJaxSCVI

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 5
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, f"adata_train_{split}.h5ad"))
adata_test = sc.read(os.path.join(output_dir, f"adata_test_{split}.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, f"adata_ood_{split}.h5ad")) 

In [4]:
n_latents = [10, 32, 64, 128, 256]
n_hiddens = [512, 1024, 2048]


df_result = pd.DataFrame(columns=['validation_loss', 'elbo_validation', 'reconstruction_loss_validation', 'kl_local_validation', 'kl_global_validation', 'disentangling_loss_validation', 'train_loss', 'rec_loss', 'kld_loss', 'elbo_train', 'reconstruction_loss_train', 'kl_local_train', 'kl_global_train', 'disentangling_loss_train', 'r2_train', 'r2_ood'])

CFJaxSCVI.setup_anndata(adata_train)

for n_latent in n_latents:
    for n_hidden in n_hiddens:
        vae = CFJaxSCVI(adata_train, gene_likelihood="normal", n_latent = n_latent, n_hidden = n_hidden)

        vae.train(
            max_epochs=2,
            batch_size=1024,
            plan_kwargs=dict(
                lr=1e-4,
            ),
            early_stopping=True,
            early_stopping_patience=20,
        )
        config = str(n_latent)+ "_"+str(n_hidden) 
        res = [vae.history[k].iloc[-1].values[0] for k in vae.history.keys()]
        #adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train)
        #adata_train.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_train, give_mean="True")
        #try:
        #    r2_train = r2_score(np.array(np.mean(adata_train.X, axis=0))[0,:], np.mean(adata_train.obsm["reconstruction"], axis=0))
        #except:
        #    r2_train = 0.0
        adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)
        adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood, give_mean="True")
        try:
            r2_ood = r2_score(np.mean(adata_ood.obsm["reconstruction"], axis=0), np.array(np.mean(adata_ood.X, axis=0))[0,:])
        except:
            r2_ood = 0.0
        r2_train = 0.0
        res.extend([r2_train, r2_ood])
        df_result.loc[config] =res
        df_result.to_csv('new_result_hyper_sciplex_split_5.csv')
        print(res)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU availabl

Epoch 1/2:   0%|          | 0/2 [00:00<?, ?it/s]

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:385: You have overridden `transfer_batch_to_device` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.


Epoch 2/2: 100%|██████████| 2/2 [00:16<00:00,  7.57s/it, v_num=1, train_loss_step=36.1, train_loss_epoch=38.3]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 2/2: 100%|██████████| 2/2 [00:16<00:00,  8.34s/it, v_num=1, train_loss_step=36.1, train_loss_epoch=38.3]
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


ValueError: cannot set a row with mismatched columns

In [10]:
df_result = pd.DataFrame(columns=['r2_ood'])


In [7]:
df_result

,validation_loss,elbo_validation,reconstruction_loss_validation,kl_local_validation,kl_global_validation,disentangling_loss_validation,train_loss,rec_loss,kld_loss,elbo_train,reconstruction_loss_train,kl_local_train,kl_global_train,disentangling_loss_train,r2_train,r2_ood


In [8]:
config

'10_512'

In [11]:
df_result.loc[config] = r2_ood

In [12]:
df_result

,r2_ood
10_512,0.94228
